In [88]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [89]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [90]:
class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        return self.disc(x)
        
        

In [91]:
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim),
            nn.Tanh(),
        )
    def forward(self, x):
        return self.gen(x)
    

In [92]:
#Hyperparameter etc,
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64
image_dim = 28 * 28 *1
batch_size = 32
num_epochs = 50

In [93]:
disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
dataset = datasets.MNIST(root = "dataset/", transform = transforms, download = True)
loader =  DataLoader(dataset, batch_size = batch_size, shuffle= True)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MNIST/real")
step = 0

for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]
        ### Train discriminator
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward(retain_graph = True)
        opt_disc.step()
        ###train generator

        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()
        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1



100%|█████████████████████████████| 9912422/9912422 [00:14<00:00, 662793.40it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|████████████████████████████████| 28881/28881 [00:00<00:00, 3441061.67it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|█████████████████████████████| 1648877/1648877 [00:04<00:00, 345618.91it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|█████████████████████████████████| 4542/4542 [00:00<00:00, 10738742.26it/s]


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw

Epoch [0/50] Batch 0/1875                       Loss D: 0.6667, loss G: 0.6922
Epoch [1/50] Batch 0/1875                       Loss D: 0.5574, loss G: 0.8833
Epoch [2/50] Batch 0/1875                       Loss D: 0.7985, loss G: 0.6249
Epoch [3/50] Batch 0/1875                       Loss D: 0.5895, loss G: 0.9813
Epoch [4/50] Batch 0/1875                       Loss D: 0.4630, loss G: 1.2900
Epoch [5/50] Batch 0/1875                       Loss D: 0.6632, loss G: 0.9299
Epoch [6/50] Batch 0/1875                       Loss D: 0.5259, loss G: 1.0765
Epoch [7/50] Batch 0/1875                       Loss D: 0.8813, loss G: 0.6925
Epoch [8/50] Batch 0/1875                       Loss D: 0.8995, loss G: 0.7979
Epoch [9/50] Batch 0/1875                       Loss D: 0.5979, loss G: 0.8868
Epoch [10/50] Batch 0/1875                       Loss D: 0.6643, loss G: 0.9876
Epoch [11/50] Batch 0/1875                       Loss